In [1]:
library(RPostgreSQL)
library(survey)
library(tidyverse)

Loading required package: DBI
Loading required package: grid
Loading required package: Matrix
Loading required package: survival

Attaching package: ‘survey’

The following object is masked from ‘package:graphics’:

    dotchart

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.8.0     ✔ stringr 1.3.0
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
data_dir <- file.path("..", "data")

In [3]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [4]:
sql <-
"select hadm_id, icustay_id, echo,
    intime, outtime, echo_time, deathtime,
    norepinephrine_max, dobutamine_flag,
    vasofreeday28, ventfreeday28,
    sofa, sofa_drop_2, sofa_drop_3,
    day1, day2, day3, down2, down3,
    lactate_diff, creatinine_diff,
    lactate_diff_24, creatinine_diff_24
from (select hadm_id, icustay_id, echo, intime, outtime, echo_time, deathtime from merged_data) co
left join subgroup using (hadm_id, icustay_id, echo)
left join lactate_diff using (hadm_id)
left join creatinine_diff using (hadm_id)
left join (select hadm_id, lactate_diff as lactate_diff_24 from lactate_diff_24) lt_24 using (hadm_id)
left join (select hadm_id, creatinine_diff as creatinine_diff_24 from creatinine_diff_24) cr_24 using (hadm_id)
left join (select icustay_id, day1, day2, day3, down2, down3 from fluid) fl using (icustay_id)"

In [5]:
subgroup <- dbGetQuery(con, sql)
head(subgroup)

hadm_id,icustay_id,echo,intime,outtime,echo_time,deathtime,norepinephrine_max,dobutamine_flag,vasofreeday28,⋯,sofa_drop_3,day1,day2,day3,down2,down3,lactate_diff,creatinine_diff,lactate_diff_24,creatinine_diff_24
163557,200003,1,2199-08-02 19:50:04,2199-08-08 17:09:18,2199-08-03,2199-10-17 00:00:00,0.000000,0,26.68750,⋯,3,1710.4062,276.9948,-769.100,1433.41146,2479.506,1.6,0.0,1.3,-0.1
127229,200014,1,2105-02-16 23:16:48,2105-02-18 16:53:29,2105-02-18,2105-02-27 00:00:00,0.000000,0,0.00000,⋯,2,1516.0000,NA,NA,NA,NA,NA,-0.1,NA,-0.2
165426,200030,1,2150-11-13 14:08:02,2150-11-19 17:51:59,2150-11-16,2152-04-05 00:00:00,1.084800,1,26.29167,⋯,5,422.5490,327.0000,-2115.000,95.54901,2537.549,-1.1,0.0,-1.1,0.0
198650,200033,0,2198-08-07 17:56:17,2198-08-21 14:59:18,NA,2198-08-21 11:15:00,0.000000,0,0.00000,⋯,-5,720.7583,346.7278,2223.795,374.03047,-1503.037,-0.1,0.1,-0.4,0.0
159243,200036,0,2181-08-26 16:38:25,2181-08-29 16:28:48,NA,2184-11-13 00:00:00,0.000000,0,28.00000,⋯,7,3750.0000,905.0000,1601.000,2845.00000,2149.000,NA,0.5,NA,0.4
132255,200075,1,2159-09-23 00:13:20,2159-09-25 01:55:17,2159-09-25,NA,1.058141,0,27.66319,⋯,5,2616.9449,-1983.8871,NA,4600.83201,NA,NA,0.1,NA,0.0


In [6]:
outcomes <- c("ventfreeday28", "vasofreeday28", "dobutamine_flag", "day1", "day2", "day3",
              "sofa_drop_2", "sofa_drop_3",
              "norepinephrine_max", "lactate_diff", "creatinine_diff",
              "lactate_diff_24", "creatinine_diff_24")
outcomes

[1] "ventfreeday28"      "vasofreeday28"      "dobutamine_flag"   
 [4] "day1"               "day2"               "day3"              
 [7] "sofa_drop_2"        "sofa_drop_3"        "norepinephrine_max"
[10] "lactate_diff"       "creatinine_diff"    "lactate_diff_24"   
[13] "creatinine_diff_24"

In [7]:
subgroup <- subgroup %>%
select(!!! rlang::syms(c("icustay_id", outcomes)))

head(subgroup)

icustay_id,ventfreeday28,vasofreeday28,dobutamine_flag,day1,day2,day3,sofa_drop_2,sofa_drop_3,norepinephrine_max,lactate_diff,creatinine_diff,lactate_diff_24,creatinine_diff_24
200003,24.20833,26.68750,0,1710.4062,276.9948,-769.100,4,3,0.000000,1.6,0.0,1.3,-0.1
200014,0.00000,0.00000,0,1516.0000,NA,NA,1,2,0.000000,NA,-0.1,NA,-0.2
200030,28.00000,26.29167,1,422.5490,327.0000,-2115.000,4,5,1.084800,-1.1,0.0,-1.1,0.0
200033,0.00000,0.00000,0,720.7583,346.7278,2223.795,0,-5,0.000000,-0.1,0.1,-0.4,0.0
200036,28.00000,28.00000,0,3750.0000,905.0000,1601.000,5,7,0.000000,NA,0.5,NA,0.4
200075,27.80903,27.66319,0,2616.9449,-1983.8871,NA,4,5,1.058141,NA,0.1,NA,0.0


In [8]:
nrow(subgroup)

[1] 6361

In [9]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE

In [10]:
full_data <- readRDS(file.path(data_dir, "full_data_ps.rds"))
head(full_data)

icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,lab_pco2_flag,lab_creatinine_flag,lab_potassium_flag,lab_troponin_flag,lab_po2_flag,lab_lactate_flag,echo_int,mort_28_day_int,ps,ps_weight
228416,159895,5491,MICU,2151-06-29 01:18:27,2151-07-27 16:00:56,1,76.58225,1,2151-06-30,⋯,1,1,1,0,1,1,1,0,0.5525232,1.809879
278148,169700,49395,MICU,2197-01-09 00:45:42,2197-01-10 06:46:29,1,82.72606,1,NA,⋯,0,1,1,0,0,0,0,1,0.2804613,1.389779
274174,103722,14855,MICU,2181-09-08 23:43:13,2181-09-26 16:52:02,1,61.84390,1,2181-09-11,⋯,1,1,1,0,1,1,1,0,0.3012020,3.320031
214954,170515,54642,MICU,2176-03-11 04:58:23,2176-03-15 18:00:48,1,63.86507,1,2176-03-11,⋯,1,1,1,1,1,1,1,0,0.7982833,1.252688
222457,183493,96815,MICU,2105-06-04 20:57:55,2105-06-06 20:01:51,1,62.84018,1,NA,⋯,0,1,1,0,0,0,0,0,0.2244447,1.289399
264413,134244,81436,MICU,2133-03-27 16:31:37,2133-04-04 16:15:28,1,62.60148,1,2133-03-31,⋯,0,1,1,0,0,1,1,0,0.3320492,3.011602


In [11]:
full_data <- full_data %>%
left_join(subgroup, by = "icustay_id") %>%
mutate(dobutamine_flag = as.factor(dobutamine_flag))
head(full_data)

icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,day1,day2,day3,sofa_drop_2,sofa_drop_3,norepinephrine_max,lactate_diff,creatinine_diff,lactate_diff_24,creatinine_diff_24
228416,159895,5491,MICU,2151-06-29 01:18:27,2151-07-27 16:00:56,1,76.58225,1,2151-06-30,⋯,-2110.000,-1930.000,-1475.000,1,0,0.000000,-0.2,-0.7,-0.2,-0.2
278148,169700,49395,MICU,2197-01-09 00:45:42,2197-01-10 06:46:29,1,82.72606,1,NA,⋯,1966.617,NA,NA,-16,8,0.000000,NA,NA,NA,NA
274174,103722,14855,MICU,2181-09-08 23:43:13,2181-09-26 16:52:02,1,61.84390,1,2181-09-11,⋯,-300.000,-1827.000,-10.000,3,2,0.000000,NA,0.5,NA,0.4
214954,170515,54642,MICU,2176-03-11 04:58:23,2176-03-15 18:00:48,1,63.86507,1,2176-03-11,⋯,4299.590,2899.645,-1283.263,4,5,7.977352,NA,0.2,0.2,0.3
222457,183493,96815,MICU,2105-06-04 20:57:55,2105-06-06 20:01:51,1,62.84018,1,NA,⋯,5988.072,2102.761,NA,0,1,0.000000,NA,0.2,NA,0.1
264413,134244,81436,MICU,2133-03-27 16:31:37,2133-04-04 16:15:28,1,62.60148,1,2133-03-31,⋯,1507.392,-1400.000,NA,2,3,2.748114,NA,-0.1,NA,0.0


In [12]:
missing <- full_data %>%
select(outcomes) %>%
summarise_all(funs(sum(is.na(.)) / n())) %>%
gather("outcome", "na")

missing

outcome,na
ventfreeday28,0.00000000
vasofreeday28,0.00000000
dobutamine_flag,0.00000000
day1,0.03615784
day2,0.15343499
day3,0.33013677
sofa_drop_2,0.00000000
sofa_drop_3,0.00000000
norepinephrine_max,0.00000000
lactate_diff,0.71608238


In [13]:
features <- data.table::fread(file.path(data_dir, "features.csv"), data.table = FALSE) %>%
pull(feature)

features
length(features)

[1] "first_careunit"             "age"                       
 [3] "gender"                     "weight"                    
 [5] "saps"                       "sofa"                      
 [7] "elix_score"                 "vent"                      
 [9] "vaso"                       "icu_adm_weekday"           
[11] "icu_adm_hour"               "icd_chf"                   
[13] "icd_afib"                   "icd_renal"                 
[15] "icd_liver"                  "icd_copd"                  
[17] "icd_cad"                    "icd_stroke"                
[19] "icd_malignancy"             "vs_heart_rate_first"       
[21] "vs_map_first"               "vs_temp_first"             
[23] "lab_hemoglobin_first"       "lab_platelet_first"        
[25] "lab_wbc_first"              "lab_ph_first"              
[27] "lab_chloride_first"         "lab_sodium_first"          
[29] "lab_bun_first"              "lab_bicarbonate_first"     
[31] "lab_pco2_first"             "lab_creatinine_first"      
[33] "lab_potassium_first"        "lab_po2_first"             
[35] "lab_lactate_first"          "sedative"                  
[37] "vs_cvp_flag"                "lab_creatinine_kinase_flag"
[39] "lab_bnp_flag"               "lab_troponin_flag"

[1] 40

In [14]:
wtd_pval <- data.table::fread(file.path(data_dir, "wtd_pval.csv"), data.table = FALSE)
features_unbalanced <- wtd_pval %>%
    filter(pval < 0.05) %>%
    pull(cov)

features_unbalanced

[1] "sofa"                       "vent"                      
 [3] "vaso"                       "sedative"                  
 [5] "icd_chf"                    "icd_afib"                  
 [7] "lab_chloride_first"         "lab_bnp_flag"              
 [9] "lab_troponin_flag"          "lab_creatinine_kinase_flag"

In [15]:
ipw_svydesign <- svydesign(ids = ~ icustay_id, weights = ~ ps_weight, data = full_data)

In [16]:
all_cov <- outcomes %>%
map_dbl(function(outcome) {
    fml <- features %>%
    c("echo", .) %>%
    paste(collapse = " + ") %>%
    sprintf("%s ~ %s", outcome, .)
    
    if (is.factor(full_data[[outcome]])) {
        mod <- svyglm(as.formula(fml), family = quasibinomial, design = ipw_svydesign)
    } else {
        mod <- svyglm(as.formula(fml), family = quasi, design = ipw_svydesign)
    }
    
    mod %>% summary %>% coefficients %>% .["echo1", 4]
}) %>%
data.frame(outcome = outcomes, pval = ., stringsAsFactors = FALSE)

all_cov

outcome,pval
ventfreeday28,1.464278e-01
vasofreeday28,1.394802e-04
dobutamine_flag,1.012378e-03
day1,8.939215e-01
day2,6.806586e-02
day3,1.777849e-04
sofa_drop_2,2.632704e-03
sofa_drop_3,5.176941e-09
norepinephrine_max,2.367474e-05
lactate_diff,1.403272e-02


In [17]:
unb_cov <- outcomes %>%
map_dbl(function(outcome) {
    fml <- features_unbalanced %>%
    c("echo", .) %>%
    paste(collapse = " + ") %>%
    sprintf("%s ~ %s", outcome, .)
    
    if (is.factor(full_data[[outcome]])) {
        mod <- svyglm(as.formula(fml), family = quasibinomial, design = ipw_svydesign)
    } else {
        mod <- svyglm(as.formula(fml), family = quasi, design = ipw_svydesign)
    }
    
    mod %>% summary %>% coefficients %>% .["echo1", 4]
}) %>%
data.frame(outcome = outcomes, pval = ., stringsAsFactors = FALSE)

unb_cov

outcome,pval
ventfreeday28,4.802945e-01
vasofreeday28,4.531889e-03
dobutamine_flag,1.846086e-05
day1,9.104450e-01
day2,2.880065e-03
day3,1.103833e-06
sofa_drop_2,4.953871e-06
sofa_drop_3,6.183852e-24
norepinephrine_max,2.852326e-08
lactate_diff,3.835708e-03


In [18]:
multivariate <- outcomes %>%
map_dbl(function(outcome) {
    fml <- features %>%
    c("echo", .) %>%
    paste(collapse = " + ") %>%
    sprintf("%s ~ %s", outcome, .)
    
    if (is.factor(full_data[[outcome]])) {
        mod <- glm(as.formula(fml), family = binomial, data = full_data)
    } else {
        mod <- glm(as.formula(fml), family = gaussian, data = full_data)
    }
    
    mod %>% summary %>% coefficients %>% .["echo1", 4]
}) %>%
data.frame(outcome = outcomes, pval = ., stringsAsFactors = FALSE)

multivariate

outcome,pval
ventfreeday28,9.651383e-02
vasofreeday28,5.794427e-05
dobutamine_flag,1.374514e-03
day1,9.354837e-01
day2,5.203845e-02
day3,1.365362e-05
sofa_drop_2,4.659638e-05
sofa_drop_3,6.415538e-12
norepinephrine_max,3.385410e-03
lactate_diff,6.352126e-03


In [19]:
tb_raw <- (all_cov %>% rename(dbwa = pval)) %>%
full_join(unb_cov %>% rename(dbwu = pval), by = "outcome") %>%
full_join(multivariate %>% rename(multi = pval), by = "outcome") %>%
full_join(missing, by = "outcome")

tb_raw

outcome,dbwa,dbwu,multi,na
ventfreeday28,1.464278e-01,4.802945e-01,9.651383e-02,0.00000000
vasofreeday28,1.394802e-04,4.531889e-03,5.794427e-05,0.00000000
dobutamine_flag,1.012378e-03,1.846086e-05,1.374514e-03,0.00000000
day1,8.939215e-01,9.104450e-01,9.354837e-01,0.03615784
day2,6.806586e-02,2.880065e-03,5.203845e-02,0.15343499
day3,1.777849e-04,1.103833e-06,1.365362e-05,0.33013677
sofa_drop_2,2.632704e-03,4.953871e-06,4.659638e-05,0.00000000
sofa_drop_3,5.176941e-09,6.183852e-24,6.415538e-12,0.00000000
norepinephrine_max,2.367474e-05,2.852326e-08,3.385410e-03,0.00000000
lactate_diff,1.403272e-02,3.835708e-03,6.352126e-03,0.71608238


In [20]:
tb_fmt <- tb_raw %>%
mutate_at(vars(dbwa, dbwu, multi), funs(case_when(
    . < 0.001 ~ "<0.001",
    TRUE ~ as.character(round(., 3))
))) %>%
rename(cov = outcome) %>%
mutate(cov = case_when(
    cov == "ventfreeday28" ~ "Ventilation free days in 28 days",
    cov == "vasofreeday28" ~ "Vasopressor free days in 28 days",
    cov == "dobutamine_flag" ~ "Dobutamine Use",
    cov == "day1" ~ "IV Fluid Day 1 (mL)",
    cov == "day2" ~ "IV Fluid Day 2 (mL)",
    cov == "day3" ~ "IV Fluid Day 3 (mL)",
    cov == "sofa_drop_2" ~ "SOFA Reduction Day 2",
    cov == "sofa_drop_3" ~ "SOFA Reduction Day 3",
    cov == "norepinephrine_max" ~ "Norepinephrine (maximum dosage mg/min)",
    cov == "lactate_diff" ~ "Serum Lactate Reduction (48 hours)",
    cov == "creatinine_diff" ~ "Serum Creatinine Reduction (48 hours)",
    cov == "lactate_diff_24" ~ "Serum Lactate Reduction (24 hours)",
    cov == "creatinine_diff_24" ~ "Serum Creatinine Reduction (24 hours)",
    TRUE ~ cov
)) %>%
mutate(na = round(na * 100, 3)) %>%
setNames(c("cov", "p value (Doubly Robust All Covariates)",
           "p value (Doubly Robust Unbalanced Covariates)",
           "p value (Multivariate)", "Missing data (%)"))

tb_fmt

cov,p value (Doubly Robust All Covariates),p value (Doubly Robust Unbalanced Covariates),p value (Multivariate),Missing data (%)
Ventilation free days in 28 days,0.146,0.48,0.097,0.000
Vasopressor free days in 28 days,<0.001,0.005,<0.001,0.000
Dobutamine Use,0.001,<0.001,0.001,0.000
IV Fluid Day 1 (mL),0.894,0.91,0.935,3.616
IV Fluid Day 2 (mL),0.068,0.003,0.052,15.343
IV Fluid Day 3 (mL),<0.001,<0.001,<0.001,33.014
SOFA Reduction Day 2,0.003,<0.001,<0.001,0.000
SOFA Reduction Day 3,<0.001,<0.001,<0.001,0.000
Norepinephrine (maximum dosage mg/min),<0.001,<0.001,0.003,0.000
Serum Lactate Reduction (48 hours),0.014,0.004,0.006,71.608


In [21]:
subgroup_tb1 <- data.table::fread(file.path(data_dir, "subgroup.csv"), data.table = FALSE)
subgroup_tb1

cov,Non-Echo,Echo,p value,SMD,Non-Echo (weighted cohort),Echo (weighted cohort),p value (IPW),SMD (weighted cohort),Non-Echo (PS matching),Echo (PS matching),SMD (PS matching),p value (PS matching),Missing data
n,3099,3262,,NA,5739.48,5842.36,,NA,1626,1626,NA,,
Ventilation free days in 28 days,19.09 (13.53),18.02 (25.13),0.033,0.053,14.78 (16.31),16.18 (36.90),0.903,0.003,18.10 (14.70),19.60 (32.64),0.059,0.089,0%
Vasopressor free days in 28 days,20.25 (12.61),20.13 (14.91),0.741,0.008,18.13 (13.84),19.86 (20.23),0.047,0.054,19.31 (12.73),20.81 (16.95),0.100,0.004,0%
Dobutamine Use,0.7%,4.2%,<0.001,0.224,1.1%,3.4%,<0.001,0.155,1.2%,2.1%,0.067,0.077,0%
IV Fluid Day 1 (mL),2303.55 (3670.67),2602.06 (4025.58),0.002,0.077,3578.20 (3784.46),3889.22 (4421.38),0.394,0.023,2552.61 (3907.32),2555.53 (3900.29),0.001,0.844,3.6%
IV Fluid Day 2 (mL),964.10 (2645.46),1334.97 (3005.02),<0.001,0.131,2293.20 (2795.53),2484.97 (3305.21),0.001,0.100,1083.61 (2867.93),1336.38 (2925.28),0.087,0.033,15.3%
IV Fluid Day 3 (mL),304.31 (2194.46),720.55 (2638.10),<0.001,0.172,1077.95 (2689.29),1683.90 (3066.03),<0.001,0.170,306.64 (2245.55),813.12 (2718.42),0.203,0.001,33%
SOFA Reduction Day 2,1.22 (4.34),0.89 (3.39),0.001,0.086,1.01 (2.83),0.55 (3.03),<0.001,0.128,1.31 (4.55),0.68 (3.26),0.159,<0.001,0%
SOFA Reduction Day 3,2.29 (4.49),1.49 (4.07),<0.001,0.186,1.22 (3.32),0.78 (3.57),<0.001,0.247,2.46 (4.76),1.15 (3.99),0.299,<0.001,0%
Norepinephrine (maximum dosage mg/min),0.81 (2.39),1.78 (5.69),<0.001,0.221,1.77 (3.26),3.05 (8.49),<0.001,0.154,1.04 (2.68),1.38 (3.13),0.117,<0.001,0%


In [22]:
subgroup_tb <- subgroup_tb1 %>%
full_join(tb_fmt, by = "cov") %>%
select(1, 2, 3, 5, 17, 6, 7, 9, 8, 15, 16, 10, 11, 12, 13)

In [23]:
subgroup_tb

cov,Non-Echo,Echo,SMD,p value (Multivariate),Non-Echo (weighted cohort),Echo (weighted cohort),SMD (weighted cohort),p value (IPW),p value (Doubly Robust All Covariates),p value (Doubly Robust Unbalanced Covariates),Non-Echo (PS matching),Echo (PS matching),SMD (PS matching),p value (PS matching)
n,3099,3262,NA,NA,5739.48,5842.36,NA,,NA,NA,1626,1626,NA,
Ventilation free days in 28 days,19.09 (13.53),18.02 (25.13),0.053,0.097,14.78 (16.31),16.18 (36.90),0.003,0.903,0.146,0.48,18.10 (14.70),19.60 (32.64),0.059,0.089
Vasopressor free days in 28 days,20.25 (12.61),20.13 (14.91),0.008,<0.001,18.13 (13.84),19.86 (20.23),0.054,0.047,<0.001,0.005,19.31 (12.73),20.81 (16.95),0.100,0.004
Dobutamine Use,0.7%,4.2%,0.224,0.001,1.1%,3.4%,0.155,<0.001,0.001,<0.001,1.2%,2.1%,0.067,0.077
IV Fluid Day 1 (mL),2303.55 (3670.67),2602.06 (4025.58),0.077,0.935,3578.20 (3784.46),3889.22 (4421.38),0.023,0.394,0.894,0.91,2552.61 (3907.32),2555.53 (3900.29),0.001,0.844
IV Fluid Day 2 (mL),964.10 (2645.46),1334.97 (3005.02),0.131,0.052,2293.20 (2795.53),2484.97 (3305.21),0.100,0.001,0.068,0.003,1083.61 (2867.93),1336.38 (2925.28),0.087,0.033
IV Fluid Day 3 (mL),304.31 (2194.46),720.55 (2638.10),0.172,<0.001,1077.95 (2689.29),1683.90 (3066.03),0.170,<0.001,<0.001,<0.001,306.64 (2245.55),813.12 (2718.42),0.203,0.001
SOFA Reduction Day 2,1.22 (4.34),0.89 (3.39),0.086,<0.001,1.01 (2.83),0.55 (3.03),0.128,<0.001,0.003,<0.001,1.31 (4.55),0.68 (3.26),0.159,<0.001
SOFA Reduction Day 3,2.29 (4.49),1.49 (4.07),0.186,<0.001,1.22 (3.32),0.78 (3.57),0.247,<0.001,<0.001,<0.001,2.46 (4.76),1.15 (3.99),0.299,<0.001
Norepinephrine (maximum dosage mg/min),0.81 (2.39),1.78 (5.69),0.221,0.003,1.77 (3.26),3.05 (8.49),0.154,<0.001,<0.001,<0.001,1.04 (2.68),1.38 (3.13),0.117,<0.001


In [24]:
data.table::fwrite(subgroup_tb, file.path(data_dir, "secondary.csv"))